In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model
from keras.applications.vgg16 import VGG16


In [2]:
img_size = 48
batch_size = 64

base_path = "C:\\Users\\arvin\\OneDrive\\Desktop\\miniproj\\"

# Load pre-trained model without classifier
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze layers of pre-trained model
for layer in vgg_model.layers:
    layer.trainable = False

# Create new classifier for transfer learning
model = Sequential()
model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Combine pre-trained model with new classifier
model = Model(inputs=vgg_model.input, outputs=model(vgg_model.output))

opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Data generator to augment data for training
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory(base_path+"train", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='rgb',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True)

# Data generator to augment data for validation
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory(base_path+"validation", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='rgb',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=False)


58889256/58889256 [==============================] - 263s 4us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                            

In [3]:
epochs = 15
steps_per_epoch= train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("new_model_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.00001,model='auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(
        x= train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks
)

model_json = model.to_json()
with open("new_model.json","w") as json_file:
    json_file.write(model_json)


Epoch 1/15
450/450 [==============================] - ETA: 0s - loss: 6.1678 - accuracy: 0.2497
Epoch 1: saving model to new_model_weights.h5
450/450 [==============================] - 336s 743ms/step - loss: 6.1678 - accuracy: 0.2497 - val_loss: 1.8500 - val_accuracy: 0.3078 - lr: 5.0000e-04
Epoch 2/15
450/450 [==============================] - ETA: 0s - loss: 1.8812 - accuracy: 0.2954
Epoch 2: saving model to new_model_weights.h5
450/450 [==============================] - 422s 938ms/step - loss: 1.8812 - accuracy: 0.2954 - val_loss: 1.6997 - val_accuracy: 0.3331 - lr: 5.0000e-04
Epoch 3/15
450/450 [==============================] - ETA: 0s - loss: 1.7291 - accuracy: 0.3189
Epoch 3: saving model to new_model_weights.h5
450/450 [==============================] - 203s 451ms/step - loss: 1.7291 - accuracy: 0.3189 - val_loss: 1.6535 - val_accuracy: 0.3594 - lr: 5.0000e-04
Epoch 4/15
450/450 [==============================] - ETA: 0s - loss: 1.6785 - accuracy: 0.3341
Epoch 4: saving model 